### Imports

In [128]:
from typing import Tuple, Optional, Deque, List
from collections import deque
from random import shuffle
from IPython.display import HTML, display
from enum import Enum

### Logic

In [130]:
class Suit(Enum):
    SPADES = ('♠', 'black')
    HEARTS = ('♥', 'red')
    CLUBS = ('♣', 'black')
    DIAMONDS = ('♦', 'red')

    def __init__(self, symbol, color):
        self.symbol = symbol
        self.color = color

class Rank(Enum):
    TWO = '2'
    THREE = '3'
    FOUR = '4'
    FIVE = '5'
    SIX = '6'
    SEVEN = '7'
    EIGHT = '8'
    NINE = '9'
    TEN = '10'
    JACK = 'J'
    QUEEN = 'Q'
    KING = 'K'
    ACE = 'A'

class Card:
    def __init__(self, rank: Rank, suit: Suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank.value}{self.suit.symbol}"

    def __eq__(self, other):
        return isinstance(other, Card) and self.rank == other.rank and self.suit == other.suit # Makes sure that other is a card object

    def __hash__(self):
        return hash((self.rank, self.suit))

class Deck:
    def __init__(self):
        self.cards = []
        self.burned = set()
        self.dealt = set()
        self._populate()

    def _populate(self):
        for suit in Suit:
            for rank in Rank:
                self.cards.append(Card(rank, suit))

    def deal(self, count=1):
        dealt = self.cards[:count] # Syntax has to do with slicing: Grabs first count cards from deck
        self.dealt.update(dealt) # Adds cards to self.dealt set (Update is extend() for sets)
        self.cards = self.cards[count:] # Removes first count cards from self.cards
        return dealt

    def burn(self):
        if self.cards:
            burn_card = self.cards.pop(0)
            self.burned.add(burn_card)
            return burn_card

    def shuffle(self):
        """Shuffles deck in place, in the future, will want a function that returns a copy for simulations."""
        # Moves sets back into the deck, and clears the sets, another way to do it would be to use the extend funciton
        # For the addition to a list
        self.cards += list(self.dealt) + list(self.burned)
        self.dealt.clear()
        self.burned.clear()
        shuffle(deck.cards)

In [131]:
POSITIONS_BY_PLAYER_COUNT = {
    2: ["Button", "Big Blind"],
    3: ["Button", "Small Blind", "Big Blind"],
    4: ["Button", "Small Blind", "Big Blind", "UTG"],
    5: ["Button", "Small Blind", "Big Blind", "UTG", "CO"],
    6: ["Button", "Small Blind", "Big Blind", "UTG", "HJ", "CO"],
    7: ["Button", "Small Blind", "Big Blind", "UTG", "MP", "HJ", "CO"],
    8: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "MP", "HJ", "CO"],
    9: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "UTG+2", "MP", "HJ", "CO"],
    10: ["Button", "Small Blind", "Big Blind", "UTG", "UTG+1", "UTG+2", "MP", "LJ", "HJ", "CO"]
}

In [132]:
class Player:
    def __init__(self, 
                 name: str, 
                 stack: float, 
                 is_bot: bool = False, 
                 seat_index: int = -1, # Unassigned until seated 
                 in_hand: bool = True, 
                 has_folded: bool = False, 
                 current_bet: float = 0.0, 
                 hole_cards: Optional[Tuple[Card, Card]] = None
            ):
        
        self.name = name
        self.stack = stack
        self.is_bot = is_bot
        self.seat_index = seat_index
        self.in_hand = in_hand
        self.current_bet = current_bet
        self.hole_cards = hole_cards
        self.position: Optional[str] = None

    def reset_for_new_hand(self):
        """Resets player attributes for a new hand. Doen't affect in_hand attribute, becuase They may not be dealt out,
           or they are spectating or something."""
        self.hole_cards = None
        self.has_folded = False
        self.current_bet = 0

In [133]:
class Position:
    def __init__(self, num_players: int):
        if not (2 <= num_players <= 10):
            raise ValueError("Only 2 to 10 players supported.")
        self.positions = POSITIONS_BY_PLAYER_COUNT[num_players]

    def get_position(self, seat_index: int, button_index: int) -> str:
        """Given a seat_index and the button's index, return the official position name."""
        offset = (seat_index - button_index) % len(self.positions)
        return self.positions[offset]

class Action(Enum):
    FOLD = 'fold'
    CALL = 'call'
    CHECK = 'check'
    BET = 'bet'
    RAISE = 'raise'

class Phase(Enum):
    WAITING = "waiting"
    PREDEAL = "predeal"
    PREFLOP = "preflop"
    FLOP = "flop"
    TURN = "turn"
    RIVER = "river"
    SHOWDOWN = "showdown"

In [134]:
class Game:
    def __init__(self,
                 small_blind: float = 0.25,
                 big_blind: float = 0.50,
                 debug = False,
                 players: Optional[List[Player]] = None
            ):
        
        self.players: Deque[Player] = deque()
        if players:
            self.add_player(players)
        self.community_cards = []
        self.dealer_index = 0
        self.pot = 0.0
        self.phase = "predeal"
        self.action_log = []
        self.deck = Deck()
        self.small_blind = small_blind
        self.big_blind = big_blind
        self.minimum_raise = big_blind
        self.debug = debug
        # Might want to add a minimum raise increment in the future here.

    def _log(self, msg):
        if self.debug:
            print(msg)

    def add_player(self, player_or_list):
        """Adds list of players, or single player to the game."""
        if isinstance(player_or_list, list):
            for p in player_or_list:
                self._add_single_player(p)

        else:
            self._add_single_player(player_or_list)

    def _add_single_player(self, player: Player):
        player.seat_index = len(self.players)
        self.players.append(player)

    def get_action_order(self) -> List[Player]:
        """Returns a list of active players in order of action for the current phase."""
        self._log("\n=== Game.get_action_order() ===\n")

        active_players = [p for p in self.players if p.in_hand and not p.has_folded]
        num_active = len(active_players)

        # Heads-up special case
        if num_active == 2:
            dealer = next(p for p in active_players if p.position == "Button")
            other = next(p for p in active_players if p.position == "Big Blind")
            self._log(f"Heads-up action order: {other.name}, {dealer.name}")
            return [other, dealer]

        # Use position mapping and rotate from big blind
        position_list = POSITIONS_BY_PLAYER_COUNT[num_active]

        if "Big Blind" not in position_list:
            raise ValueError("Big Blind position not found in POSITION_BY_PLAYER_COUNT.")

        bb_index = position_list.index("Big Blind")
        rotated_positions = position_list[bb_index + 1:] + position_list[:bb_index + 1]
        self._log(f"Rotated position order: {rotated_positions}")

        ordered_players = []
        for pos in rotated_positions:
            for p in self.players:
                if p.position == pos and p.in_hand and not p.has_folded:
                    ordered_players.append(p)

        self._log("Action order:")
        for p in ordered_players:
            self._log(f"{p.name} ({p.position})")
        return ordered_players

    def start_new_hand(self):
        self._log("\n=== Game.start_new_hand() ===\n")
        
        self.rotate_dealer()
        self.reset_players() # This isn't built yet
        self.assign_player_positions()
        self.post_blinds()
        self.deck.shuffle()
        
        self.community_cards = []
        self.pot = 0.0
        self.phase = Phase.PREFLOP
        self.deal_hole_cards()
        self.run_betting_round(Phase.PREFLOP)

    def execute_hand(self):
        self.start_new_hand()

        self.deal_flop()
        self.run_betting_round(Phase.FLOP)

        self.deal_turn()
        self.run_betting_round(Phase.TURN)

        self.deal_river()
        self.run_betting_round(Phase.RIVER)

        self._log("\n=== Showdown (not implemented yet) ===")
        self.action_log.append("Showdowm occurs.")

    def rotate_dealer(self):
        """Rotate The dealer"""
        n = len(self.players)
        for i in range(1, n + 1):
            next_index = (self.dealer_index + i) % n
            if self.players[next_index].in_hand:
                self.dealer_index = next_index

    def assign_player_positions(self):
        """Assign official position names (e.g. Button, SB, BB)
           to all active players based on dealer index."""
        self._log("\n=== Game.assign_player_positions() ===\n")

        active_players = [p for p in self.players if p.in_hand]
        num_active = len(active_players)

        self._log(f"Assigning positions with {num_active} active players. Dealer index: {self.dealer_index}")

        if num_active < 2:
            raise ValueError("Need at least 2 players in hand to assign positions.")

        position_helper = Position(num_active)

        self._log(f"Position list: {position_helper.positions}")

        seat_to_player = {player.seat_index: player for player in active_players}
        self._log("Seat-to-player map:")
        for seat, p in seat_to_player.items():
            self._log(f"  Seat {seat}: {p.name}")

        ordered_seats = []
        for i in range(num_active):
            seat = (self.dealer_index + i) % len(self.players)
            if seat in seat_to_player:
                ordered_seats.append(seat)

        self._log(f"Ordered seat indices: {ordered_seats}")

        for offset, seat_index in enumerate(ordered_seats):
            player = seat_to_player[seat_index]
            position_name = position_helper.positions[offset]
            player.position = position_name
            self._log(f"{player.name} (seat {seat_index}) -> {position_name}")

    def post_blinds(self):
        """Deduct small and big blinds from the appropriate players and add to the pot.
           In heads-up, the button is small blind and the other player is big blind.
           If a player has less than the blind, they are forced all-in.
        """
        self._log("\n=== Game.post_blinds() ===")
    
        small_blind_player = None
        big_blind_player = None
    
        active_players = [p for p in self.players if p.in_hand]
    
        if len(active_players) < 2:
            raise ValueError("Cannot post blinds with fewer than 2 players.")
    
        # Heads-up special case
        if len(active_players) == 2:
            for player in active_players:
                if player.position == "Button":
                    small_blind_player = player
                elif player.position == "Big Blind":
                    big_blind_player = player
        else:
            for player in active_players:
                if player.position == "Small Blind":
                    small_blind_player = player
                elif player.position == "Big Blind":
                    big_blind_player = player
    
        if not small_blind_player or not big_blind_player:
            raise ValueError("Could not identify both small and big blind players.")
    
        # Post small blind
        sb_amt = min(self.small_blind, small_blind_player.stack)
        small_blind_player.stack -= sb_amt
        small_blind_player.current_bet = sb_amt
        self.pot += sb_amt
        self.action_log.append(f"{small_blind_player.name} posts small blind of {sb_amt}.")
        self._log(f"{small_blind_player.name} (seat {small_blind_player.seat_index}) posts SMALL blind: {sb_amt}")
    
        # Post big blind
        bb_amt = min(self.big_blind, big_blind_player.stack)
        big_blind_player.stack -= bb_amt
        big_blind_player.current_bet = bb_amt
        self.pot += bb_amt
        self.action_log.append(f"{big_blind_player.name} posts big blind of {bb_amt}.")
        self._log(f"{big_blind_player.name} (seat {big_blind_player.seat_index}) posts BIG blind: {bb_amt}")
    
        self._log(f"Total pot after blinds: {self.pot}")



    def reset_players(self):
        for player in self.players:
            player.reset_for_new_hand()


    def deal_hole_cards(self):
        for player in self.players:
            if player.in_hand:
                player.hole_cards = tuple(self.deck.deal(2))
                self.action_log.append(f"{player.name} is dealt hole cards.")
        

    def run_preflop(self):
        """I think this might be depricated now? not sure if depricated is the right word but might not need to be used."""
        self._log("\n=== Game.run_preflop() ===\n")
        action_order = self.get_action_order()

        for player in action_order:
            if player.has_folded or not player.in_hand:
                continue

            call_amount = self.get_call_amount(player)
            
            # "Simulated logic for now"?
            if player.stack >= call_amount:
                player.stack -= call_amount
                player.current_bet += call_amount
                self.pot += call_amount
                self._log(f"{player.name} calls {call_amount}")
                self.action_log.append(f"{player.name} calls {call_amount}")
            else:
                player.has_folded = True
                self._log(f"{player.name} folds")
                self.action_log.append(f"{player.name} folds")


    def get_call_amount(self, player: Player) -> float:
        """Returns how much the player needs to call to match the highest current bet."""
        self._log("\n=== Game.get_call_amount() ===\n")
        highest_bet = max(p.current_bet for p in self.players if p.in_hand and not p.has_folded)
        return max(0.0, highest_bet - player.current_bet)

    def deal_flop(self):
        self._log("\n=== Game.deal_flop()===\n")
        self.deck.burn()
        self.community_cards.extend(self.deck.deal(3))
        self.phase = Phase.FLOP
        self._log(f"Flop: {' '.join(str(c) for c in self.community_cards)}")

    def deal_turn(self):
        self._log("\n=== Game.deal_turn() ===\n")
        self.deck.burn()
        self.community_cards.append(self.deck.deal(1)[0])
        self.phase = Phase.TURN
        self._log(f"Turn: {self.community_cards[-1]}")

    def deal_river(self):
        self._log("\n=== Game.deal_river() ===\n")
        self.deck.burn()
        self.community_cards.append(self.deck.deal(1)[0])
        self.phase = Phase.RIVER
        self._log(f"River: {self.community_cards[-1]}")

    def run_betting_round(self, phase: Phase):
        self._log(f"\n=== Game.run_betting_round({phase.value}) ===\n")
        action_order = self.get_action_order()
        highest_bet = max(p.current_bet for p in self.players if p.in_hand)

        players_to_act = action_order.copy()
        already_acted = set()

        while players_to_act:
            player = players_to_act.pop(0)

            if player.has_folded or not player.in_hand or player.stack == 0:
                continue

            call_amount = self.get_call_amount(player)
            action, amount = self.get_player_action(player, phase, call_amount)

            if action == Action.FOLD:
                player.has_folded = True
                self.action_log.append(f"{player.name} folds.")
                self._log(f"{player.name} folds.")
            elif action == Action.CALL:
                call_amt = min(call_amount, player.stack)
                player.stack -= call_amt
                player.current_bet += call_amt
                self.pot += call_amt
                self.action_log.append(f"{player.name} calls {call_amt}.")
                self._log(f"{player.name} calls {call_amt}.")
            elif action == Action.RAISE:
                raise_amt = amount
                total_to_call = call_amount + raise_amt
                total_bet = min(total_to_call, player.stack)
                player.stack -= total_bet
                player.current_bet += total_bet
                self.pot += total_bet
                self.action_log.append(f"{player.name} raises to {player.current_bet}.")
                self._log(f"{player.name} raises to {player.current_bet}.")

                # Reset players_to_act for new betting round
                players_to_act = [p for p in self.get_action_order() if p!= player and p.in_hand and not p.has_folded]
                already_acted = set()

            already_acted.add(player)
        self._log(f"Pot after {phase.value}: {self.pot}")

    def get_player_action(self, player: Player, phase: Phase, call_amount: float) -> Tuple[Action, Optional[float]]:
        """Stub: returns a simple auto-call or fold action."""
        if player.stack <= call_amount:
            return (Action.CALL, None)
        if call_amount == 0:
            return (Action.CHECK, None)
        return (Action.CALL, None)

    def show_community_cards(self):
        return ' '.join(str(card) for card in self.community_cards)

    def end_hand_cleanup(self):
        self._log("\n Game.end_hand_cleanup() ===\n")

        for player in self.players:
            player.current_bet = 0
        self._log("Player bets reset.")

### Printing Logic

In [136]:
def styled_card_html(card):
    """Return an HTML-formatted card string with styling for Jupyter."""
    base_style = ("display:inline-block; font-family:monospace; font-size:1.2em; "\
                 "background:white; color:black; padding: 2px 6px; border-radius: 4px;"
    )

    suit_color = "red" if card.suit.color == "red" else "black"
    return f'<span style="{base_style}">{card.rank.value}<span style="color:{suit_color}">{card.suit.symbol}</span></span>'

Display Function to group cards by suit

In [138]:
def styled_deck_html(deck):
    lines = []

    for suit in Suit:
        # Collect all cards of this suit
        suit_cards = [c for c in deck.cards + list(deck.dealt) + list(deck.burned) if c.suit == suit]

        # Sort by rank using Rank enum order
        suit_cards.sort(key=lambda c: list(Rank).index(c.rank))

        line = ''
        for card in suit_cards:
            # Color logic
            if card in deck.burned:
                bg = "lightcoral"
                text = "black"
            elif card in deck.dealt:
                bg = "lightblue"
                text = "black"
            else:
                bg = "white"
                text = "black"

            style = (
                f"background:{bg}; color:{text}; display:inline-block; "
                f"font-family:monospace; font-weight:bold; font-size:1.1em; "
                f"padding:2px 6px; border-radius:4px; margin:1px;"
            )

            suit_color = "red" if card.suit.color == "red" else text
            html = f'<span style="{style}">{card.rank.value}<span style="color:{suit_color}">{card.suit.symbol}</span></span>'
            line += html

        lines.append(f'<div style="margin-bottom:15px;">{line}</div>')

    return '<div style="line-height: 2;">' + ''.join(lines) + '</div>'


### Testing

Testing a game

Testing Deck 5/11/25

In [142]:
deck = Deck() # Seems to be ok

In [143]:
players = [Player("Sam", 100), Player("Jamie", 100), Player("Steve", 100)]
game = Game(players=players, debug=True)

In [190]:
game.execute_hand()


=== Game.start_new_hand() ===


=== Game.assign_player_positions() ===

Assigning positions with 3 active players. Dealer index: 0
Position list: ['Button', 'Small Blind', 'Big Blind']
Seat-to-player map:
  Seat 0: Sam
  Seat 1: Jamie
  Seat 2: Steve
Ordered seat indices: [0, 1, 2]
Sam (seat 0) -> Button
Jamie (seat 1) -> Small Blind
Steve (seat 2) -> Big Blind

=== Game.post_blinds() ===
Jamie (seat 1) posts SMALL blind: 0.25
Steve (seat 2) posts BIG blind: 0.5
Total pot after blinds: 1.5

=== Game.run_betting_round(preflop) ===


=== Game.get_action_order() ===

Rotated position order: ['Button', 'Small Blind', 'Big Blind']
Action order:
Sam (Button)
Jamie (Small Blind)
Steve (Big Blind)

=== Game.get_call_amount() ===

Sam calls 0.5.

=== Game.get_call_amount() ===

Jamie calls 0.25.

=== Game.get_call_amount() ===

Pot after preflop: 0.75

=== Game.deal_flop()===

Flop: 10♠ 2♥ J♦

=== Game.run_betting_round(flop) ===


=== Game.get_action_order() ===

Rotated position order: ['But

In [145]:
second_card = Card(Rank.JACK, Suit.CLUBS)
card = Card(Rank.NINE, Suit.HEARTS)
display(HTML(styled_card_html(second_card)))
display(HTML(styled_card_html(card)))

In [146]:
deck = Deck()

In [147]:
deck.shuffle()

In [148]:
dealt_card = deck.deal()
burn_card = deck.burn()
dealt2 = deck.deal()
burn2 = deck.burn()

In [149]:
display(HTML(styled_deck_html(deck)))

### Ramdom & Depricated

In [151]:
# Adding doc string would be cool.
Card?

Init signature: Card(rank: __main__.Rank, suit: __main__.Suit)
Docstring:      <no docstring>
Type:           type
Subclasses:     

### Notes

<span style="color: green">5/3/2025</span>

Working on the Deck class currently. Have to implement hash and eq special methods to make it work in a set? Then i will need to create a display function to group cards by suit. uses lambda to sort. i am not all that familiar with that. I also imported display from IPython.display, which i did not have before

<span style="color: red">Dunder Eq and Hash</span>

dunder eq & hash come into play when doing some_set = {card1, card2} so python can tell if they are the same.

eq defines equality between objects.

hash enables objects to use in sets and dicts: Python requires objects in sets/dicts to have a <b>hash value</b>: Number that stays the same as long as the object's value doesnt change. (Unique value for sets and maps)

<span style="color: red">Lambda functions</span>

<span style="color: red">IPython.display</span>

<span style="color: green">5/3/2025</span>

Worked on getting the deck to display with burnt & dealt cards. Next want to work on a shuffle feature I think. It might be an easy change to get the deck to display with Ace's first. that is really just a small thing. Then I will eventually need to add players, gameflow, position, calculating, hands and pot odds. 

<span style="color: green">5/4/2025</span>

<span style="color: red">Git & GitHub</span>

<span style="color: red">Uploading</span>

Uploaded to Git & Github

- git clone https://github.com/your-username/your-repo.git
- cd your-repo
- mv /path/to/your_notebook.ipynb .
- git add your_notebook.ipynb
- git commit -m "Add my notebook"
- git push origin main

Now I have a local git repository in my /poker folder linked to GitHub.

If I were to move the entire folder (repository) everything works as long as I 

- navigate to new location to run Git commands
- Git only works inside repo directory (Cant push/ pull from outside of it.)

Once in Repo folder

- git add "my_notebook"
- git commit -m "Add some message"
- git push origin main

<span style="color: Red">Branching</span>

- git checkout -b new-feature
- make changes ....
- git add . (or file, cause I dont want to commit the database.)
- git commit -m "Message"
- git push origin main new-feature (push new branch to github, but for a solo project, terminal is faster.)

<span style="color: Red">Merging</span>

- git checkout main
- git pull origin main (make sure it's up to date.)
- git merge new-feature

If that Succeeds...

- git push origin main (Update github with merged result)
- git branch -d new-feature (delete local branch)
- git push origin --delete new-feature (delete remote branch)

Or use GitHub Interface...

- Push your branch: git push origin new-feature
- Go to GitHub, click "Compare & Pull Request"
- Review and click "Merge"
- Delete branch via button

<span style="color: Red">Stashing</span>

Like puting your changes in a drawer and coming back to them later...

- git stash save "WIP: notebook update" (stash with a message)
- git stash list (list al stashes)
- git stash pop stash@{1} (restore specific stash)
- git stash drop stash@{0} (discard stash if no longer need it

<span style="color: Red">Naming Convention</span>

_populate() or _funcition() means to say that: "This is an implementaiton detail, and you probably shouldn't use it directly."

<span style="color: green">5/5/2025</span>

<span style="color: red">Game, Players, Position....</span>

<span style="color: red">Modulo Arithmetic</span>

Finding the remainder when one number is divided by another. Often used for <b>Wraparound Logic</b> for example...

(11 + 2) % 12 (adding two hours at 11pm to a 12 hour clock.)


<span style="color: green">5/6/2025</span>

Make classes more robust with optional defaults.

<span style="color: green">5/11/2025</span>

Started with a review of everything.

Game.post_blinds()

Game.assign_player_positions()

Continued working on the flow for creating a poker game.

<span style="color: green">5/12/2025</span>

<span style="color: green">5/13/2025</span>

Game Flow

Things to work on...

Take some notes on this tomorrow.